# Problema 5: Producción del Sector Agro

Un terreno de 1798 hectáreas será utilizado para plantar café, soja, maíz, naranja y caña de azúcar. Cada producto genera una ganancia especifica.
  
<img src="productos_table.png" width= 300>

**Con el objetivo de maximizar la ganancia total**, indique cual es la mejor manera de distribuir las tierras para la plantación de los productos, indicando el porcentaje de terreno referente a la producción de cada producto.

**Restricciones**:
- La plantación de naranjas y caña de azúcar juntas deben ocupar un área mínima de 800 hectáreas.
- La plantación de naranjas no puede pasar un área de 10 hectáreas.
- La plantación de maíz debe ocupar un área mínima de 360 hectáreas. 
- La plantación de café debe ocupar un área mínima de 180 hectáreas. 
- Las plantaciones de maíz, soja y café juntas no pueden pasar un área de 899 hectáreas.
- Total hectáreas disponibles 1798

## Variables de decisión

* $x_{0}: \text{Hectareas de Café}$
* $x_{1}: \text{Hectareas de Soja}$
* $x_{2}: \text{Hectareas de Maiz}$
* $x_{3}: \text{Hectareas de Naranja}$
* $x_{4}: \text{Hectareas de Caña de Azucar}$

* $\text{Función Objetivo} = 120*x_{0} + 160*x_{1} + 75*x_{2} + 140*x_{3} + 140*x_{4}$.

## Restricciones

* $x_{3} + x_{4} >= 800$
* $x_{3} <= 10$
* $x_{2} >= 360$
* $x_{0} >= 180$
* $x_{0} + x_{1} + x_{2} <= 899$
* $x_{0} + x_{1} + x_{2} + x_{3} + x_{4} <= 1798$

## Instalación de Paquetes

In [225]:
# https://deap.readthedocs.io/en/master/
!pip install deap

In [226]:
# Bibliotecas a serem utilizadas
import random
import numpy as np
from deap import algorithms, base, creator, tools

## Preparación para Optimización

1. **Función objetivo** 
2. **Variables de decisión**
3. **Operadores**

In [227]:
# Función Objetivo - Calculo de la ganancia
def objetive_function(individual):
  #codigo de la función
  #resultado retorna como tupla (result)
  return (individual[0]*120 + individual[1]*160 + individual[2]*75 + individual[3]*140 + individual[4]*140),

## Restricciones

- La plantación de naranjas y caña de azúcar juntas deben ocupar un área mínima de 800 hectáreas.
- La plantación de naranjas no puede pasar un área de 10 hectáreas.
- La plantación de maíz debe ocupar un área mínima de 360 hectáreas. 
- La plantación de café debe ocupar un área mínima de 180 hectáreas. 
- Las plantaciones de maíz, soja y café juntas no pueden pasar un área de 899 hectáreas.
- Total hectáreas disponibles 1798

* $x_{3} + x_{4} >= 800$
* $x_{3} <= 10$
* $x_{2} >= 360$
* $x_{0} >= 180$
* $x_{0} + x_{1} + x_{2} <= 899$
* $x_{0} + x_{1} + x_{2} + x_{3} + x_{4} <= 1798$

In [228]:
#Ref: https://deap.readthedocs.io/en/master/tutorials/advanced/constraints.html?highlight=feasible#penalty-function  
def feasible(individual):
  if (individual[3] + individual[4]) < 800:
    return False
  if individual[3] > 10:
    return False
  if individual[2] < 360:
    return False
  if individual[0] < 180:
    return False
  if (individual[0] + individual[1] + individual[2]) > 899:
    return False
  if (individual[0] + individual[1] + individual[2] + individual[3] + individual[4]) > 1798:
    return False
  return True 

In [229]:
# Creación estructura de fitness e individuo
##para problema de maximización el peso es positivo y minimización el peso es negativo
creator.create("FitnessMax", base.Fitness, weights=(1.0,)) 
creator.create("Individual", list, fitness=creator.FitnessMax)

In [230]:
toolbox = base.Toolbox()

# Generador de atributos reales: nombre, función que genera cada variable, intervalo (limite superior e inferior)
toolbox.register("attr_int", random.randint,0,1000) #genera los numeros entre 0 y 1000

# Generador de individuo
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, 5)

# Generar la población
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


In [231]:
#Inicializar Operadores: https://deap.readthedocs.io/en/master/api/tools.html
toolbox.register("evaluate",objetive_function)

# Crear Restricciones
toolbox.decorate("evaluate", tools.DeltaPenalty(feasible, -1000000))

toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=1000, indpb=0.05)
toolbox.register("select",tools.selTournament, tournsize=3)

In [232]:
pop = toolbox.population(n=100)                # inicialização da pop
hof = tools.HallOfFame(1)                      # melhor indivíduo
stats = tools.Statistics(lambda ind:ind.fitness.values) # estatísticas
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

In [233]:
print(f"Individuos de la población generada: \n{pop[:5]}")
print("len: ", len(pop))

Individuos de la población generada: 
[[694, 373, 493, 789, 704], [313, 839, 369, 692, 528], [106, 247, 684, 607, 473], [760, 205, 693, 484, 478], [705, 533, 689, 973, 896]]
len:  100


# Ciclo evolutivo

In [234]:
pop, log =algorithms.eaSimple(population=pop, 
                              toolbox=toolbox, 
                              cxpb=0.5, 
                              mutpb=0.1, 
                              ngen=100, 
                              stats=stats, 
                              halloffame=hof, 
                              verbose=1)

gen	nevals	avg   	std	min   	max   
0  	100   	-1e+06	0  	-1e+06	-1e+06
1  	46    	-1e+06	0  	-1e+06	-1e+06
2  	52    	-1e+06	0  	-1e+06	-1e+06
3  	55    	-1e+06	0  	-1e+06	-1e+06
4  	54    	-1e+06	0  	-1e+06	-1e+06
5  	54    	-1e+06	0  	-1e+06	-1e+06
6  	72    	-1e+06	0  	-1e+06	-1e+06
7  	44    	-1e+06	0  	-1e+06	-1e+06
8  	56    	-1e+06	0  	-1e+06	-1e+06
9  	56    	-1e+06	0  	-1e+06	-1e+06
10 	49    	-1e+06	0  	-1e+06	-1e+06
11 	50    	-1e+06	0  	-1e+06	-1e+06
12 	48    	-1e+06	0  	-1e+06	-1e+06
13 	58    	-1e+06	0  	-1e+06	-1e+06
14 	47    	-1e+06	0  	-1e+06	-1e+06
15 	62    	-1e+06	0  	-1e+06	-1e+06
16 	51    	-1e+06	0  	-1e+06	-1e+06
17 	45    	-1e+06	0  	-1e+06	-1e+06
18 	63    	-1e+06	0  	-1e+06	-1e+06
19 	59    	-1e+06	0  	-1e+06	-1e+06
20 	55    	-1e+06	0  	-1e+06	-1e+06
21 	49    	-1e+06	0  	-1e+06	-1e+06
22 	78    	-1e+06	0  	-1e+06	-1e+06
23 	45    	-1e+06	0  	-1e+06	-1e+06
24 	59    	-1e+06	0  	-1e+06	-1e+06
25 	57    	-1e+06	0  	-1e+06	-1e+06
26 	56    	-1e+06	0  	-1e+06

# Resultados

In [235]:
# Mejor Solución
print(f"Mejor Individuo: {hof[0]}\n")
cafe = hof[0][0]
soja = hof[0][1]
maiz = hof[0][2]
naranja = hof[0][3]
Cana_azucar = hof[0][4]
total_hectareas = (sum(hof[0]))

print(f"Total Hectareas de Café: {cafe}")
print(f"Total Hectareas de Soja: {soja}")
print(f"Total Hectareas de Maiz: {maiz}")
print(f"Total Hectareas de Naranja: {naranja}")
print(f"Total Hectareas de Caña de Azucar: {Cana_azucar}")
print(f"Total Hectareas Cultivadas: {total_hectareas}\n")

print(f"Restricciones:")
print(f"Naranja + Caña de Azucar >= 800 : {(naranja + Cana_azucar)>=800}")
print(f"Naranja <= 10 : {(naranja)<=10}")
print(f"Maíz >= 360 : {(maiz)>=360}")
print(f"Café >= 180 : {(cafe)>=180}")
print(f"Maíz + Soja + Café <= 899 : {(maiz + soja + cafe)<= 899}")
print(f"Total hectáreas cultivadas <= 1789 : {(total_hectareas)<= 1789}\n")

#validacióon del mejor individuo
print(f"Lucro Total: ${objetive_function(hof[0])[0]}")

Mejor Individuo: [694, 373, 493, 789, 704]

Total Hectareas de Café: 694
Total Hectareas de Soja: 373
Total Hectareas de Maiz: 493
Total Hectareas de Naranja: 789
Total Hectareas de Caña de Azucar: 704
Total Hectareas Cultivadas: 3053

Restricciones:
Naranja + Caña de Azucar >= 800 : True
Naranja <= 10 : False
Maíz >= 360 : True
Café >= 180 : True
Maíz + Soja + Café <= 899 : False
Total hectáreas cultivadas <= 1789 : False

Lucro Total: $388955


In [236]:
a = [276, 258, 360, 1, 901]
feasible(a)

True

In [237]:
# Individuo Viable?
feasible(hof[0])

False